In [1]:
def job_feature (rpt_dt):
    import pandas as pd
    import numpy as np
    import sys
    #from datetime import datetime
    import time
    from oraDB import oraDB
    from sqlalchemy import text
    #conn,cur= oraDB.connect()
   #now = datetime.now()
    #start_time = now.strftime("%H:%M:%S")
    start_time = time.time()
    dbEngine = oraDB.create_engine()
    with dbEngine.connect() as conn:
        #creating tables
        tbl_nm = 'CINS_FEATURE_STORE_TEST'
        #ALTER SESSION
        conn.execute(text("ALTER SESSION FORCE PARALLEL DML PARALLEL 64"))
        conn.execute(text("ALTER SESSION FORCE PARALLEL QUERY PARALLEL 64"))
        try:
            create_tbl = open('script_create_tbl.txt','r')
            query_create = create_tbl.read()
        except:
            print("Unable to read Create Table text file")
            conn.close()
            sys.exit()
            create_tbl.close()
        n = 0
        print('--------STARTING CREATING TABLE(S)--------')
        for sql_create_tbl in query_create.split(';'):
            sql_create_tbl = sql_create_tbl.replace("{RPT_DT}",rpt_dt)
            n = n + 1
            print(sql_create_tbl)
            try:
                create_time = time.time()
                conn.execute(text(sql_create_tbl))
                conn.execute(text("COMMIT"))
                print('-----Finished creating ' + str(n) + ' table(s)-----')
                print('Runtime: ', time.time() - create_time)
            except:
                print("Unable to create table(s)")
                conn.close()
                sys.exit()
        #insert features
        try:
            insert = open('script_insert_main.txt','r') 
            query_insert = insert.read()
        except:
            print("Unable to read Insert Feature text file")
            conn.close()
            sys.exit()    
            insert.close()
        m = 0
        print('--------STARTING INSERTING FEATURE(S)--------')
        #Delete 
        sql_del = f"DELETE FROM {tbl_nm} WHERE RPT_DT = '{rpt_dt}'"
        try:
            print(sql_del)
            del_time = time.time()
            conn.execute(text(sql_del))
            conn.execute(text("COMMIT"))
            print(f'-----Sucessfully deleted data in {tbl_nm} with report date = {rpt_dt}-----')
            print('Runtime: ', time.time() - del_time)
        except:
            print("Unable to delete feature(s)")
            conn.close()
            sys.exit()
            drop.close()
        #Insert
        for sql_insert in query_insert.split(';'):    
            sql_insert = sql_insert.replace("{RPT_DT}",rpt_dt) 
            sql_insert = sql_insert.replace("{TBL_NM}",tbl_nm)
            m = m + 1
            print(sql_insert)
            try:
                insert_time = time.time()
                insert_result = conn.execute(text(sql_insert))
                conn.execute(text("COMMIT"))
                print(f"""{insert_result.rowcount} rows inserted!""")
                print('-----Finished inserting ' + str(m) + ' feature(s)-----')
                print('Runtime: ', time.time() - insert_time)
            except:
                print("Unable to insert feature(s)")
                conn.close()
                sys.exit()
        #drop tmp tables
        try:
            drop = open('script_drop_tbl.txt','r') 
            query_drop = drop.read()
        except:
            print("Unable to read Drop Table text file")
            conn.close()
            sys.exit()
            drop.close()
        t = 0
        print('--------STARTING DROPING TABLE(S)--------')
        for sql_drop in query_drop.split(';'):

            sql_drop = sql_drop.replace("{RPT_DT}",rpt_dt)

            t = t + 1
            print(sql_drop)
            try:
                drop_time = time.time()
                conn.execute(text(sql_drop))
                conn.execute(text("COMMIT"))
                print('-----Finished dropping ' + str(t) + ' table(s)-----')
                print('Runtime: ', time.time() - drop_time)
            except:
                print("Unable to drop table(s)")
                conn.close()
                sys.exit()
    print('DONE!!!')
    end_time = time.time()
    #print('Job finished at: ', end_time)
    print('Total runtime: ', end_time - start_time  )
    #close files and connection
    create_tbl.close()
    insert.close()
    drop.close()
    conn.close()


In [3]:
job_feature('18-10-2023')

--------STARTING CREATING TABLE(S)--------
CREATE TABLE CINS_TMP_CUST AS
SELECT A.CUSTOMER_CDE FROM 
 (SELECT customer_cde FROM dw_analytics.dw_customer_dim
 WHERE SUB_SECTOR_CDE IN ('1700','1602') AND ACTIVE = '1' AND COMPANY_KEY = '1') A 
 JOIN (select distinct customer_cde from dw_analytics.dw_cust_product_loc_fct
 WHERE CUST_STATUS = 'HOAT DONG' AND PROCESS_DT = TO_DATE('18-10-2023','DD-MM-YYYY')) B 
 ON A.CUSTOMER_CDE = B.CUSTOMER_CDE
Unable to create table(s)


AttributeError: 'tuple' object has no attribute 'tb_frame'